# Attention Toxicity Model

This notebook trains a model to detect toxicity in online comments. It uses a CNN architecture for text classification trained on the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) and pre-trained GloVe embeddings which can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/).

This model is a modification of [example code](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) found in the [Keras Github repository](https://github.com/fchollet/keras) and released under an [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE). For further details of this license, find it [online](https://github.com/fchollet/keras/blob/master/LICENSE) or in this repository in the file KERAS_LICENSE. 

## Usage Instructions
(TODO: nthain) - Move to README

Prior to running the notebook, you must:

* Download the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973)
* Download pre-trained [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* (optional) To skip the training step, you will need to download a model and tokenizer file. We are looking into the appropriate means for distributing these (sometimes large) files.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd


In [50]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K


def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    Tx = 250
    repeator = RepeatVector(Tx)
    concatenator = Concatenate(axis=-1)
    densor1 = Dense(10, activation = "tanh")
    densor2 = Dense(1, activation = "relu")
    activator = Activation('softmax') # We are using a custom softmax(axis = 1) loaded in this notebook
    dotor = Dot(axes = 1)
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context

In [60]:
from model_tool import (DEFAULT_EMBEDDINGS_PATH,
DEFAULT_MODEL_DIR, DEFAULT_HPARAMS, ToxModel)
from keras.utils import to_categorical
from keras.layers import Multiply
from keras.callbacks import TensorBoard


tensorboard = TensorBoard(log_dir="logs/atn_lstm_v1",write_graph=True)


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import cPickle
import json
import os
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import Input
from keras.layers import MaxPooling1D
from keras.models import load_model
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn import metrics
import datetime

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

class AttentionToxModel(ToxModel):
    def __init__(self,
                 model_name=None,
                 model_dir=DEFAULT_MODEL_DIR,
                 embeddings_path=DEFAULT_EMBEDDINGS_PATH,
                 hparams=None):
        self.model_dir = model_dir
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.hparams = DEFAULT_HPARAMS.copy()
        self.embeddings_path = embeddings_path
        if hparams:
            self.update_hparams(hparams)
        if model_name:
            self.load_model_from_name(model_name)
            self.load_probs_model(model_name)
        self.print_hparams()

    def load_probs_model(self, model_name):
        probs_model_name = model_name + "_probs"
        self.probs_model = load_model(
                os.path.join(
                    self.model_dir, '%s_model.h5' % probs_model_name))

    def save_prob_model(self):
        self.probs_model_name = self.model_name + "probs"

    def build_dense_attention_layer(self, input_tensor):
        # softmax
        attention_probs = Dense(self.hparams['max_sequence_length'],
                                activation='softmax',
                                name='attention_vec')(input_tensor)
        # context vector
        attention_mul = Multiply()([input_tensor, attention_probs])
        return {'attention_probs': attention_probs,
                'attention_preds': attention_mul}

    def train(
                self,
                training_data_path,
                validation_data_path, text_column,
                label_column, model_name):
        self.model_name = model_name
        self.save_hparams(model_name)

        train_data = pd.read_csv(training_data_path)
        valid_data = pd.read_csv(validation_data_path)

        print('Fitting tokenizer...')
        self.fit_and_save_tokenizer(train_data[text_column])
        print('Tokenizer fitted!')

        print('Preparing data...')
        train_text, train_labels = (self.prep_text(train_data[text_column]),
                                    to_categorical(train_data[label_column]))
        valid_text, valid_labels = (self.prep_text(valid_data[text_column]),
                                    to_categorical(valid_data[label_column]))
        print('Data prepared!')

        print('Loading embeddings...')
        self.load_embeddings()
        print('Embeddings loaded!')

        print('Building model graph...')
        self.build_model()
        print('Training model...')

        preds_save_path = os.path.join(
                            self.model_dir, '%s_model.h5' % self.model_name)
        probs_save_path = os.path.join(
                            self.model_dir, '%s_probs_model.h5'
                            % self.model_name)
        preds_callbacks = [ModelCheckpoint(
                            preds_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose']), tensorboard]
        probs_callbacks = [ModelCheckpoint(
                            probs_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose']),tensorboard]

        if self.hparams['stop_early']:
            early_stop = EarlyStopping(
                            min_delta=self.hparams['es_min_delta'],
                            monitor='val_loss',
                            patience=self.hparams['es_patience'],
                            verbose=self.hparams['verbose'], mode='auto')
            probs_callbacks.append(early_stop)
            preds_callbacks.append(early_stop)

        self.model.fit(train_text,
                       train_labels,
                       batch_size=self.hparams['batch_size'],
                       epochs=self.hparams['epochs'],
                       validation_data=(valid_text, valid_labels),
                       callbacks=preds_callbacks,
                       verbose=2)

        print('Model trained!')
        print('Best model saved to {}'.format(preds_save_path))
        print('Fitting probs model')

        self.probs_model.fit(
                    train_text,
                    train_labels,
                    batch_size=self.hparams['batch_size'],
                    epochs=self.hparams['epochs'],
                    validation_data=(valid_text, valid_labels),
                    callbacks=probs_callbacks,
                    verbose=2)
        self.probs_model = load_model(probs_save_path)
        print('Loading best model from checkpoint...')
        self.model = load_model(preds_save_path)
        print('Model loaded!')

    def build_model(self):
        print('print inside build model')
        sequence_input = Input(
                            shape=(self.hparams['max_sequence_length'],),
                            dtype='int32')
        embedding_layer = Embedding(
                            len(self.tokenizer.word_index) + 1,
                            self.hparams['embedding_dim'],
                            weights=[self.embedding_matrix],
                            input_length=self.hparams['max_sequence_length'],
                            trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
    
        # Defined shared layers as global variables
        Tx = self.hparams['max_sequence_length']

        """
        x = Bidirectional(LSTM(128,return_sequences=True))(x)
        
        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'])(x)
        # TODO(nthain): Parametrize the number and size of fully connected layers
        x = Dense(128, activation='relu')(x)
        preds = Dense(2, activation='softmax')(x)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        """
        X = x
        n_a = 32
        n_s = 128
        post_activation_LSTM_cell = LSTM(128, return_state = True)
        output_layer = Dense(self.hparams['max_sequence_length'], activation='softmax')
        s0 = Input(shape=(n_s,), name='s0')
        c0 = Input(shape=(n_s,), name='c0')
        s = s0
        c = c0
        
        
         # Initialize empty list of outputs
        outputs = []
    
        ### START CODE HERE ###
        
    
        # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
        a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
        # Step 2: Iterate for Ty steps
        for t in range(self.hparams['max_sequence_length']):
    
            # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
            context = one_step_attention(a, s)

            # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
            # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
            s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])

            # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
            out = output_layer(s)

            # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
            outputs.append(out)

            # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
        model = Model(inputs=[sequence_input,s0,c0], outputs=outputs)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = model
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=rmsprop,
                metrics=['acc'])
        """
        self.model = Model(sequence_input, preds)
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=rmsprop,
                metrics=['acc'])
        """
        
        
        """
        for filter_size, kernel_size, pool_size in zip(
                self.hparams['cnn_filter_sizes'],
                self.hparams['cnn_kernel_sizes'],
                self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'], name="Dropout")(x)
        x = Dense(250, activation='relu', name="Dense_RELU")(x)

        # build prediction model
        attention_dict = self.build_dense_attention_layer(x)
        preds = attention_dict['attention_preds']
        preds = Dense(2, name="preds_dense", activation='softmax')(preds)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=rmsprop,
                metrics=['acc'])
                
                
        # now make probs model
        probs = attention_dict['attention_probs']
        probs = Dense(2, name='probs_dense')(probs)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.probs_model = Model(sequence_input, preds)
        self.probs_model.compile(
                loss='mse', optimizer=rmsprop, metrics=['acc'])
        # build probabilities model
        self.save_prob_model()

        """
        


## Load Data

In [61]:
SPLITS = ['train', 'dev', 'test']

wiki = {}
debias = {}
random = {}
for split in SPLITS:
    wiki[split] = '../data/wiki_%s.csv' % split
    debias[split] = '../data/wiki_debias_%s.csv' % split
    random[split] = '../data/wiki_debias_random_%s.csv' % split

## Train Models

In [62]:
hparams = {'epochs': 20}

### Random model

In [63]:
# bidirectional
MODEL_NAME = 'atn_lstm_v1'
debias_random_model = AttentionToxModel(hparams=hparams)
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...


ValueError: The model expects 3 input arrays, but only received one array. Found: array with shape (99157, 250)

In [7]:
MODEL_NAME = 'bd_atn_random_tox_v1'
debias_random_model = AttentionToxModel(hparams=hparams)
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 5
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/5
Epoch 00000: val_loss improved from inf to 0.17158, saving model to ../models/bd_debias_random_tox_v1_model.h5
235s - loss: 0.2125 - acc: 0.9236 - val_loss: 0.1716 - val_acc: 0.9381
Epoch 2/5
Epoch 00001: val_loss improved from 0.17158 to 0.15761, saving model to ../models/bd_debias_random_tox_v1_model.h5
234s - loss: 0.1658 - acc: 0.9387 - val_loss: 0.1576 - val_acc: 0.9422
Epoch 3/5
Epoch 00002: val_loss improv

AttributeError: AttentionToxModel instance has no attribute 'probs_model'

In [ ]:
MODEL_NAME = '20_atn_cnn_debias_random_tox_v3'
debias_random_model = AttentionToxModel(hparams=hparams)
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23986, saving model to ../models/20_atn_cnn_debias_random_tox_v3_model.h5
105s - loss: 0.3014 - acc: 0.9066 - val_loss: 0.2399 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.23986 to 0.20081, saving model to ../models/20_atn_cnn_debias_random_tox_v3_model.h5
107s - loss: 0.2208 - acc: 0.9125 - val_loss: 0.2008 - val_acc: 0.9265
Epoch 3/20
Epoch 00

In [ ]:
random_test = pd.read_csv(random['test'])
debias_random_model.score_auc(random_test['comment'], random_test['is_toxic'])

### Plain wikipedia model

In [7]:
MODEL_NAME = 'atn_cnn_wiki_tox_v3'
wiki_model = AttentionToxModel(hparams=hparams)
wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 4
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/4
Epoch 00000: val_loss improved from inf to 0.23601, saving model to ../models/atn_cnn_wiki_tox_v3_model.h5
100s - loss: 0.3048 - acc: 0.9020 - val_loss: 0.2360 - val_acc: 0.9045
Epoch 2/4
Epoch 00001: val_loss improved from 0.23601 to 0.19425, saving model to ../models/atn_cnn_wiki_tox_v3_model.h5
103s - loss: 0.2152 - acc: 0.9170 - val_loss: 0.1943 - val_acc: 0.9311
Epoch 3/4
Epoch 00002: val_loss improved from 

In [8]:
wiki_test = pd.read_csv(wiki['test'])
wiki_model.score_auc(wiki_test['comment'], wiki_test['is_toxic'])

0.93516059425530385

### Debiased model

In [10]:
MODEL_NAME = 'atn_cnn_debias_tox_v3'
debias_model = ToxModel(hparams=hparams)
debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 4
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/4
Epoch 00000: val_loss improved from inf to 0.16953, saving model to ../models/atn_cnn_debias_tox_v3_model.h5
103s - loss: 0.2364 - acc: 0.9184 - val_loss: 0.1695 - val_acc: 0.9387
Epoch 2/4
Epoch 00001: val_loss improved from 0.16953 to 0.14930, saving model to ../models/atn_cnn_debias_tox_v3_model.h5
106s - loss: 0.1640 - acc: 0.9407 - val_loss: 0.1493 - val_acc: 0.9456
Epoch 3/4
Epoch 00002: val_loss improved from 0.14930 to 0.13845, s

In [11]:
debias_test = pd.read_csv(debias['test'])
debias_model.score_auc(debias_test['comment'], debias_test['is_toxic'])

0.95149490074750576